In [1]:
import sys
from pathlib import Path
from os import getcwd
import codecs

from datetime import date, timedelta

sys.path.append(str(Path(getcwd()).parent))

from os.path import join
import pandas as pd
pd.set_option('display.max_columns', None)
from sys import exit
from datetime import datetime

from bulletin.casos_confirmados import CasosConfirmados
from bulletin.tb_pacientes import TbPacientes
from bulletin.utils.normalize import normalize_hash, normalize_labels, normalize_text, date_hash, normalize_number
from bulletin.utils.static import Municipios
from bulletin.utils import static
from bulletin.utils.xls_ import fit_cols
from bulletin import default_input

# from bulletin.services.metabase import Metabase
# mb = Metabase()
# mb.download_query('old_diario')

date_parser = lambda x: pd.to_datetime(x, errors='coerce', format='%d/%m/%Y')


root: /home/vinnylg/projects/covid19datascience/bulletin
parent: /home/vinnylg/projects/covid19datascience
default_input: /home/vinnylg/projects/covid19datascience/input
default_output: /home/vinnylg/projects/covid19datascience/output


In [2]:
municipios = Municipios()
municipios['mun_resid'] = municipios['municipio'].apply(normalize_text)
municipios.loc[municipios['uf']!='PR','mun_resid'] = municipios.loc[municipios['uf']!='PR','municipio'].apply(normalize_text) + '/' + municipios['uf']

In [3]:
casosct = pd.read_pickle('casosct_22_05_2021.pkl')[['identificacao','id_notifica','uf_resid','ibge_resid','ibge_atend','nome','sexo','idade','laboratorio','dt_diag','comunicacao','is','evolucao','data_evolucao','data_com_evolucao','hash','hash_less','hash_more','hash_diag']]

notifica = pd.read_csv(join(default_input,'bancoNotifica.csv'),
                       sep=',',
                       converters = {
                           'paciente': normalize_text,
                           'origem': lambda x: normalize_number(x,fill=0),
                           'nome_mae': normalize_text,
                           'idade': lambda x: normalize_number(x,fill=0),
                           'pais_residencia': lambda x: normalize_number(x,fill=0),
                           'uf_residencia': lambda x: normalize_number(x,fill=0),
                           'ibge_residencia': lambda x: normalize_number(x,fill=999999),
                           'uf_unidade_notifica': lambda x: normalize_number(x,fill=0),
                           'ibge_unidade_notifica': lambda x: normalize_number(x,fill=999999),
                           'criterio_classificacao': lambda x: normalize_number(x,fill=0),
                           'evolucao': lambda x: normalize_number(x,fill=3) }
                       )

notifica['data_cura_obito'] = notifica['data_cura_obito'].apply(date_parser)
notifica['data_liberacao'] = notifica['data_liberacao'].apply(date_parser)

notifica.loc[notifica['evolucao']==5, 'evolucao'] = 2

notifica['hash'] = (notifica['paciente'].apply(normalize_hash) +
                  notifica['idade'].astype(str) +
                  notifica['ibge_residencia'].astype(str) )


notifica['hash'] = (notifica['paciente'].apply(normalize_hash) +
                  notifica['idade'].astype(str) +
                  notifica['ibge_residencia'].astype(str) )

notifica['hash_less'] = ( notifica['paciente'].apply(normalize_hash) +
                        notifica['idade'].apply(lambda x: str(x - 1)) +
                        notifica['ibge_residencia'].astype(str) )

notifica['hash_more'] = ( notifica['paciente'].apply(normalize_hash) +
                        notifica['idade'].apply(lambda x: str(x + 1)) +
                        notifica['ibge_residencia'].astype(str) )

notifica['hash_diag'] = notifica['paciente'].apply(normalize_hash) + notifica['data_liberacao'].apply(date_hash)

notifica = pd.merge(notifica.rename(columns={'ibge_residencia':'ibge'}),municipios[['ibge','rs','mun_resid','uf','municipio']],on='ibge',how='left').rename(columns={'ibge':'ibge_residencia'})
notifica = pd.merge(notifica.rename(columns={'ibge_unidade_notifica':'ibge'}),municipios[['ibge','mun_resid']].rename(columns={'mun_resid':'mun_atend'}),on='ibge',how='left').rename(columns={'ibge':'ibge_unidade_notifica'})

notifica

,id,origem,data_notificacao,updated_at,paciente,sexo,nome_mae,idade,data_nascimento,pais_residencia,uf_residencia,ibge_residencia,uf_unidade_notifica,ibge_unidade_notifica,exame,criterio_classificacao,data_liberacao,data_1o_sintomas,evolucao,data_cura_obito,hash,hash_less,hash_more,hash_diag,rs,mun_resid,uf,municipio,mun_atend
0,73785,1,22/04/2020,20/05/2021,ADRIANA CANDIDO,F,ANALIA CANDIDO,43,27/05/1977,1,41,410370,41,410370,"COVID-19, Biologia Molecular",1,2020-09-10,18/04/2020,1,2020-11-15,ADRIANACANDIDO43410370,ADRIANACANDIDO42410370,ADRIANACANDIDO44410370,ADRIANACANDIDO10092020,17,CAMBE,PR,Cambé,CAMBE
1,74768,1,23/04/2020,22/05/2021,CARMEM CELIA DA COSTA PAULA,F,DARCI ARAUJO DA COSTA,56,01/10/1964,1,41,411370,41,411370,Teste Rápido Antígeno (Imunocromatografia),1,NaT,15/04/2020,3,NaT,CARMEMCELIADACOSTAPAULA56411370,CARMEMCELIADACOSTAPAULA55411370,CARMEMCELIADACOSTAPAULA57411370,CARMEMCELIADACOSTAPAULA9999999,17,LONDRINA,PR,Londrina,LONDRINA
2,75615,1,23/04/2020,23/05/2021,DOMINGAS BASSO ZANELLA,M,NETILDES BERTOLINE BASSO,75,17/06/1945,1,41,410480,41,410480,Vírus Respiratório,1,2020-04-04,26/03/2020,1,2020-04-18,DOMINGASBASSOZANELLA75410480,DOMINGASBASSOZANELLA74410480,DOMINGASBASSOZANELLA76410480,DOMINGASBASSOZANELLA04042020,10,CASCAVEL,PR,Cascavel,CASCAVEL
3,81567,1,29/04/2020,21/05/2021,EDILAINE VIDAL,F,LEONI MAGAROLI VIDAL,53,30/11/1967,1,41,410690,41,410690,Teste Rápido Antígeno (Imunocromatografia),1,NaT,12/05/2021,3,NaT,EDILAINEVIDAL53410690,EDILAINEVIDAL52410690,EDILAINEVIDAL54410690,EDILAINEVIDAL9999999,2,CURITIBA,PR,Curitiba,CURITIBA
4,95247,4,04/04/2020,21/05/2021,MAGDA CRISTINA URBANEJA,F,NAO INFORMADO,57,13/07/1963,1,41,411370,0,999999,Vírus Respiratório,1,2020-04-05,30/03/2020,1,2020-04-12,MAGDACRISTINAURBANEJA57411370,MAGDACRISTINAURBANEJA56411370,MAGDACRISTINAURBANEJA58411370,MAGDACRISTINAURBANEJA05042020,17,LONDRINA,PR,Londrina,IGNORADO/EXTERIOR/IG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24652,3817610,1,23/05/2021,23/05/2021,SIDINEI VITOR,M,MARIA ORMINDA DE JESUS,42,27/01/1979,1,41,410430,41,410430,Teste Rápido Antígeno (Imunocromatografia),1,NaT,NaN,3,NaT,SIDINEIVITOR42410430,SIDINEIVITOR41410430,SIDINEIVITOR43410430,SIDINEIVITOR9999999,11,CAMPO MOURAO,PR,Campo Mourão,CAMPO MOURAO
24653,3817611,10,22/05/2021,NaN,LAERCIO JOSE PUPIO,M,MARGARIDA LEMES PUPIO,65,15/09/1955,1,41,411520,41,411520,Teste Rápido Antígeno (Imunocromatografia),1,NaT,21/05/2021,3,NaT,LAERCIOJOSEPUPIO65411520,LAERCIOJOSEPUPIO64411520,LAERCIOJOSEPUPIO66411520,LAERCIOJOSEPUPIO9999999,15,MARINGA,PR,Maringá,MARINGA
24654,3817612,1,23/05/2021,23/05/2021,THALIS DOS SANTOS,M,MARIA CRISTINA CORDEIRO DOS SANTOS,28,30/10/1992,1,41,410940,41,410940,Teste Rápido Antígeno (Imunocromatografia),1,NaT,18/05/2021,3,NaT,THALISDOSSANTOS28410940,THALISDOSSANTOS27410940,THALISDOSSANTOS29410940,THALISDOSSANTOS9999999,5,GUARAPUAVA,PR,Guarapuava,GUARAPUAVA
24655,3817620,1,23/05/2021,23/05/2021,HENRIQUE STAREPRAVO VAZ,M,ANA CLAUDIA STAREPRAVO VAZ,9,14/09/2011,1,41,412550,41,412550,Teste Rápido Antígeno (Imunocromatografia),1,2021-05-23,20/05/2021,3,NaT,HENRIQUESTAREPRAVOVAZ9412550,HENRIQUESTAREPRAVOVAZ8412550,HENRIQUESTAREPRAVOVAZ10412550,HENRIQUESTAREPRAVOVAZ23052021,2,SAO JOSE DOS PINHAIS,PR,São José dos Pinhais,SAO JOSE DOS PINHAIS


In [4]:
# aaa = notifica.copy()
notifica = notifica.loc[~(
    (notifica['data_liberacao'].isna()) |
    (notifica['sexo'].isna())
)]

notifica = notifica.loc[(
    (notifica['data_liberacao'].apply(date_parser) < pd.to_datetime(datetime.today())) &
    (notifica['data_liberacao'].apply(date_parser) > pd.to_datetime(datetime.strptime('12/03/2020','%d/%m/%Y')))
)]
notifica
print(len(notifica))

16030


In [5]:
novos_casos = notifica.loc[~(
    (notifica['hash'].isin(casosct['hash'])) |
    (notifica['hash'].isin(casosct['hash_less'])) |
    (notifica['hash'].isin(casosct['hash_more'])) |
    (notifica['hash_diag'].isin(casosct['hash_diag']))
)].copy()

novos_casos['data_com'] = pd.to_datetime(date.today()).strftime('%d/%m/%Y')
novos_casos = novos_casos.sort_values('paciente')
novos_casos[['id','paciente','sexo','idade','mun_resid', 'mun_atend', 'rs', 'exame','data_liberacao','data_com','data_1o_sintomas','hash']].to_excel('novos_casos.xlsx')
# novos_casos.to_excel('novos_casos.xlsx')
novos_casos

,id,origem,data_notificacao,updated_at,paciente,sexo,nome_mae,idade,data_nascimento,pais_residencia,uf_residencia,ibge_residencia,uf_unidade_notifica,ibge_unidade_notifica,exame,criterio_classificacao,data_liberacao,data_1o_sintomas,evolucao,data_cura_obito,hash,hash_less,hash_more,hash_diag,rs,mun_resid,uf,municipio,mun_atend,data_com
23877,3813133,1,22/05/2021,22/05/2021,ABDON ALVES DA ROCHA,M,IRACI CASAGRANDE DA ROCHA,51,12/12/1969,1,41,411760,41,411760,Teste Rápido Antígeno (Imunocromatografia),1,2021-05-22,17/05/2021,3,NaT,ABDONALVESDAROCHA51411760,ABDONALVESDAROCHA50411760,ABDONALVESDAROCHA52411760,ABDONALVESDAROCHA22052021,7,PALMAS,PR,Palmas,PALMAS,23/05/2021
13218,3739044,1,19/05/2021,22/05/2021,ABEL DA APARECIDA PEDROSO,M,IRACI DOS SANTOS PEDROSO,52,23/10/1968,1,41,412150,41,999999,"COVID-19, Biologia Molecular",1,2021-05-21,14/05/2021,3,NaT,ABELDAAPARECIDAPEDROSO52412150,ABELDAAPARECIDAPEDROSO51412150,ABELDAAPARECIDAPEDROSO53412150,ABELDAAPARECIDAPEDROSO21052021,4,REBOUCAS,PR,Rebouças,IGNORADO/EXTERIOR/IG,23/05/2021
23867,3813086,1,22/05/2021,22/05/2021,ADAO A CAMPOS,M,OLIVIA A SANTOS,81,15/06/1939,1,41,411760,41,411760,Teste Rápido Antígeno (Imunocromatografia),1,2021-05-22,19/05/2021,3,NaT,ADAOACAMPOS81411760,ADAOACAMPOS80411760,ADAOACAMPOS82411760,ADAOACAMPOS22052021,7,PALMAS,PR,Palmas,PALMAS,23/05/2021
16726,3775417,1,20/05/2021,23/05/2021,ADAO AVELINO PEREIRA,M,ANTONIA AVELINO PEREIRA,56,24/12/1964,1,41,410670,41,410670,"COVID-19, Biologia Molecular",1,2021-05-23,16/05/2021,3,NaT,ADAOAVELINOPEREIRA56410670,ADAOAVELINOPEREIRA55410670,ADAOAVELINOPEREIRA57410670,ADAOAVELINOPEREIRA23052021,14,CRUZEIRO DO SUL,PR,Cruzeiro do Sul,CRUZEIRO DO SUL,23/05/2021
23876,3813126,1,22/05/2021,22/05/2021,ADAO FERNANDES SOBRINHO,M,OLIMPIA FERNANDES,61,22/10/1959,1,41,411470,41,412810,Teste Rápido IgM (Imunocromatografia),1,2021-05-22,15/05/2021,3,NaT,ADAOFERNANDESSOBRINHO61411470,ADAOFERNANDESSOBRINHO60411470,ADAOFERNANDESSOBRINHO62411470,ADAOFERNANDESSOBRINHO22052021,12,MARIA HELENA,PR,Maria Helena,UMUARAMA,23/05/2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22155,3805442,1,22/05/2021,23/05/2021,YURI GABRIEL MENDES DE OLIVEIRA,M,FRANCINI MENDES,20,20/07/2000,1,41,410940,41,410940,"COVID-19, Biologia Molecular",1,2021-05-23,NaN,3,NaT,YURIGABRIELMENDESDEOLIVEIRA20410940,YURIGABRIELMENDESDEOLIVEIRA19410940,YURIGABRIELMENDESDEOLIVEIRA21410940,YURIGABRIELMENDESDEOLIVEIRA23052021,5,GUARAPUAVA,PR,Guarapuava,GUARAPUAVA,23/05/2021
10860,3712006,1,17/05/2021,23/05/2021,ZELIA DE FATIMA REJEI,F,ILDA GEJUNI REJEI,50,03/08/1970,1,41,410940,41,410940,"COVID-19, Biologia Molecular",1,2021-05-23,15/05/2021,3,NaT,ZELIADEFATIMAREJEI50410940,ZELIADEFATIMAREJEI49410940,ZELIADEFATIMAREJEI51410940,ZELIADEFATIMAREJEI23052021,5,GUARAPUAVA,PR,Guarapuava,GUARAPUAVA,23/05/2021
13404,3741044,1,19/05/2021,22/05/2021,ZENAIDE NOGUEIRA,F,ERNESTA ROSSI NOGUEIRA,50,21/07/1970,1,41,410660,41,410660,"COVID-19, Biologia Molecular",1,2021-05-21,15/05/2021,3,NaT,ZENAIDENOGUEIRA50410660,ZENAIDENOGUEIRA49410660,ZENAIDENOGUEIRA51410660,ZENAIDENOGUEIRA21052021,12,CRUZEIRO DO OESTE,PR,Cruzeiro do Oeste,CRUZEIRO DO OESTE,23/05/2021
22260,3806108,1,22/05/2021,22/05/2021,ZENI ALVES DE MORAES,F,ELVIRA ALVES DE,52,20/07/1968,1,41,410480,41,410480,Teste Rápido Antígeno (Imunocromatografia),1,2021-05-12,07/05/2021,1,2021-05-22,ZENIALVESDEMORAES52410480,ZENIALVESDEMORAES51410480,ZENIALVESDEMORAES53410480,ZENIALVESDEMORAES12052021,10,CASCAVEL,PR,Cascavel,CASCAVEL,23/05/2021


In [6]:
obitos_notifica = notifica.loc[notifica['evolucao']==2]
obitos_casos = casosct.loc[casosct['evolucao']==2]

novos_obitos = obitos_notifica.loc[~(
    (obitos_notifica['hash'].isin(obitos_casos['hash'])) |
    (obitos_notifica['hash'].isin(obitos_casos['hash_less'])) |
    (obitos_notifica['hash'].isin(obitos_casos['hash_more'])) |
    (obitos_notifica['hash_diag'].isin(obitos_casos['hash_diag'])) 
)]

novos_obitos = novos_obitos.loc[novos_obitos['data_cura_obito'].notna()]
novos_obitos = novos_obitos.loc[novos_obitos['hash'].isin(casosct['hash'].tolist() + novos_casos['hash'].tolist())]
# novos_obitos = novos_obitos.loc[~novos_obitos['paciente'].isin(['ANTONIO ALVES MAIA FILHO','ISRAEL PADILHA','JOAO PEREIRA','LUCILA APARECIDA BONIFACIO RIBEIRO','OSVALDO BRASIL DA CRUZ'])]
novos_obitos = novos_obitos.sort_values('paciente')

novos_obitos[['id','paciente','sexo','idade','mun_resid', 'rs', 'data_cura_obito','hash']].to_excel('novos_obitos.xlsx')
# novos_obitos.to_excel('novos_obitos.xlsx')
novos_obitos

,id,origem,data_notificacao,updated_at,paciente,sexo,nome_mae,idade,data_nascimento,pais_residencia,uf_residencia,ibge_residencia,uf_unidade_notifica,ibge_unidade_notifica,exame,criterio_classificacao,data_liberacao,data_1o_sintomas,evolucao,data_cura_obito,hash,hash_less,hash_more,hash_diag,rs,mun_resid,uf,municipio,mun_atend
4421,3539031,1,06/05/2021,23/05/2021,ADIRCE LECH,F,MARIA PEREIRA WAGNER,76,01/03/1945,1,41,412820,41,412820,"COVID-19, Biologia Molecular",1,2021-05-05,28/04/2021,2,2021-05-22,ADIRCELECH76412820,ADIRCELECH75412820,ADIRCELECH77412820,ADIRCELECH05052021,6,UNIAO DA VITORIA,PR,União da Vitória,UNIAO DA VITORIA
9675,3695012,1,17/05/2021,23/05/2021,ALBERTINA PEREIRA DA SILVA,F,JOANA MARIA DE JESUS,89,03/05/1932,1,41,410640,41,410640,"COVID-19, Biologia Molecular",1,2021-05-22,15/05/2021,2,2021-05-23,ALBERTINAPEREIRADASILVA89410640,ALBERTINAPEREIRADASILVA88410640,ALBERTINAPEREIRADASILVA90410640,ALBERTINAPEREIRADASILVA22052021,18,CORNELIO PROCOPIO,PR,Cornélio Procópio,CORNELIO PROCOPIO
9759,3696296,1,17/05/2021,23/05/2021,APARECIDA ROSA IANI,F,MARIA DA PENHA FARIAS,66,11/09/1954,1,41,411370,41,411370,"COVID-19, Biologia Molecular",1,2021-05-17,10/05/2021,2,2021-05-21,APARECIDAROSAIANI66411370,APARECIDAROSAIANI65411370,APARECIDAROSAIANI67411370,APARECIDAROSAIANI17052021,17,LONDRINA,PR,Londrina,LONDRINA
5623,3586803,1,10/05/2021,23/05/2021,BRAZ DE PAULA PACHECO,M,FRANCISCA MAXIMINA DE JESUS,85,13/01/1936,1,41,410010,41,410010,"COVID-19, Biologia Molecular",1,2021-05-11,06/05/2021,2,2021-05-22,BRAZDEPAULAPACHECO85410010,BRAZDEPAULAPACHECO84410010,BRAZDEPAULAPACHECO86410010,BRAZDEPAULAPACHECO11052021,18,ABATIA,PR,Abatiá,ABATIA
6656,3613340,1,11/05/2021,22/05/2021,CELIA RIZELLO BALDIN,F,LUIZA DELAZZARI RIZELLO,59,15/02/1962,1,41,411850,41,411850,"COVID-19, Biologia Molecular",1,2021-04-29,18/04/2021,2,2021-05-22,CELIARIZELLOBALDIN59411850,CELIARIZELLOBALDIN58411850,CELIARIZELLOBALDIN60411850,CELIARIZELLOBALDIN29042021,7,PATO BRANCO,PR,Pato Branco,PATO BRANCO
2854,3315366,2,16/04/2021,22/05/2021,EUNICE APARECIDA DE CASTRO SILVA,F,MARIA APARECIDA DE OLIVEIRA CASTRO,55,20/03/1966,1,41,410690,41,410690,"COVID-19, Biologia Molecular",1,2021-04-19,12/04/2021,2,2021-05-15,EUNICEAPARECIDADECASTROSILVA55410690,EUNICEAPARECIDADECASTROSILVA54410690,EUNICEAPARECIDADECASTROSILVA56410690,EUNICEAPARECIDADECASTROSILVA19042021,2,CURITIBA,PR,Curitiba,CURITIBA
4010,3508270,1,04/05/2021,22/05/2021,IDAZINA DA LUZ PRESTES,F,MARIA DA LUZ AUGUSTA,62,09/11/1958,1,41,411007,41,412170,Teste Rápido Antígeno (Imunocromatografia),1,2021-05-04,01/05/2021,2,2021-05-01,IDAZINADALUZPRESTES62411007,IDAZINADALUZPRESTES61411007,IDAZINADALUZPRESTES63411007,IDAZINADALUZPRESTES04052021,21,IMBAU,PR,Imbaú,RESERVA
5024,3567891,1,07/05/2021,23/05/2021,IDEMAR BARAO,M,MARIA MARTINS BARAO,55,07/01/1966,1,41,410720,41,410720,"COVID-19, Biologia Molecular",1,2021-05-09,03/05/2021,2,2021-05-23,IDEMARBARAO55410720,IDEMARBARAO54410720,IDEMARBARAO56410720,IDEMARBARAO09052021,8,DOIS VIZINHOS,PR,Dois Vizinhos,DOIS VIZINHOS
6787,3617539,10,11/05/2021,23/05/2021,JOANNE MARCEL,F,JOLIMENE JASMIN,31,11/02/1990,1,41,411520,41,411520,"COVID-19, Biologia Molecular",1,2021-05-13,07/05/2021,2,2021-05-22,JOANNEMARCEL31411520,JOANNEMARCEL30411520,JOANNEMARCEL32411520,JOANNEMARCEL13052021,15,MARINGA,PR,Maringá,MARINGA
3735,3479163,1,02/05/2021,23/05/2021,JOAO BATISTA GARCIA,M,MERCEDES MARIA KITADA GARCIA,49,27/06/1971,1,41,410110,41,410110,"COVID-19, Biologia Molecular",1,2021-05-04,25/04/2021,2,2021-05-21,JOAOBATISTAGARCIA49410110,JOAOBATISTAGARCIA48410110,JOAOBATISTAGARCIA50410110,JOAOBATISTAGARCIA04052021,18,ANDIRA,PR,Andirá,ANDIRA


In [7]:
casos_confirmados = pd.merge(casosct.rename(columns={'ibge_resid':'ibge'}),municipios[['ibge','rs']],how='left',on='ibge')

casos_confirmadosPR = casos_confirmados.loc[casos_confirmados['rs']!=99]

obitos_confirmados =  casos_confirmados.loc[casos_confirmados['evolucao']==2]

obitos_confirmadosPR = obitos_confirmados.loc[obitos_confirmados['rs']!=99]

print(f"Total de casos: {len(casos_confirmados)} + {len(novos_casos)}")
print(f"Total de obitos: {len(obitos_confirmados)} + {len(novos_obitos)}\n\n")

novos_casosPR = novos_casos.loc[novos_casos['rs']!=99].copy()
print(f"Total de casos PR: {len(casos_confirmadosPR)} + {len(novos_casosPR)}")

novos_obitosPR = novos_obitos.loc[novos_obitos['rs']!=99].copy()
print(f"Total de obitos PR: {len(obitos_confirmadosPR)} + {len(novos_obitosPR)}")

novos_casosFora = novos_casos.loc[novos_casos['rs']==99].copy()
print(f"Total de casos Fora: {len(casos_confirmados) - len(casos_confirmadosPR)} + {len(novos_casosFora)}")

novos_obitosFora = novos_obitos.loc[novos_obitos['rs']==99].copy()
print(f"Total de obitos Fora: {len(obitos_confirmados) - len(obitos_confirmadosPR)} + {len(novos_obitosFora)}")


novos_obitosPR_group = novos_obitosPR.groupby(by='municipio')

today = datetime.today()
ontem = today - timedelta(1)
anteontem = ontem - timedelta(1)
data_retroativos = ontem - timedelta(14)

retroativos = novos_casosPR.loc[(novos_casosPR['data_liberacao'].apply(date_parser) <= data_retroativos)].sort_values(by='data_liberacao')
last2weeks = novos_casosPR.loc[(novos_casosPR['data_liberacao'].apply(date_parser) > data_retroativos) & (novos_casosPR['data_liberacao'].apply(date_parser) <= anteontem)].sort_values(by='data_liberacao')
hoje = novos_casosPR.loc[(novos_casosPR['data_liberacao'].apply(date_parser) > anteontem)].sort_values(by='data_liberacao')

obitos_retroativos = novos_obitosPR.loc[(novos_obitosPR['data_cura_obito'].apply(date_parser) <= data_retroativos)].sort_values(by='data_cura_obito')
obitos_last2weeks = novos_obitosPR.loc[(novos_obitosPR['data_cura_obito'].apply(date_parser) > data_retroativos) & (novos_obitosPR['data_cura_obito'].apply(date_parser) <= anteontem)].sort_values(by='data_cura_obito')
obitos_hoje = novos_obitosPR.loc[(novos_obitosPR['data_cura_obito'].apply(date_parser) > anteontem)].sort_values(by='data_cura_obito')


with codecs.open(f"relatorio_{(today.strftime('%d/%m/%Y_%Hh').replace('/','_').replace(' ',''))}.txt","w","utf-8-sig") as relatorio:
    relatorio.write(f"{today.strftime('%d/%m/%Y')}\n")
    relatorio.write(f"{len(novos_casosPR):,} novos casos residentes ".replace(',','.'))

    if len(novos_casosFora) > 0:
        relatorio.write(f"e {len(novos_casosFora):,} não residente{'s' if len(novos_casosFora) > 1 else ''} ".replace(',','.'))
    relatorio.write(f"divulgados no PR.\n")

    relatorio.write(f"{len(casos_confirmadosPR)+len(novos_casosPR):,} casos confirmados residentes do PR.\n".replace(',','.'))
    relatorio.write(f"{len(casos_confirmados)+len(novos_casos):,} total geral.\n\n".replace(',','.'))
    relatorio.write(f"{len(novos_obitosPR):,} Óbitos residentes do PR:\n".replace(',','.'))

    for municipio, obitos in novos_obitosPR_group:
        relatorio.write(f"{len(obitos):,} {municipio}\n".replace(',','.'))

    if len(novos_obitosFora) > 0:
        relatorio.write('\n')
        relatorio.write(f"{len(novos_obitosFora):,} Óbito{'s' if len(novos_obitosFora) > 1 else ''} não residente{'s' if len(novos_obitosFora) > 1 else ''} do PR.\n".replace(',','.'))

    relatorio.write('\n')
    relatorio.write(f"{len(obitos_confirmadosPR)+len(novos_obitosPR):,} óbitos residentes do PR.\n".replace(',','.'))
    relatorio.write(f"{len(obitos_confirmados)+len(novos_obitos):,} total geral.\n\n".replace(',','.'))

    for _, row in novos_obitos.iterrows():
       relatorio.write(f"{row['sexo']}\t{row['idade']}\t{row['municipio'] if row['rs'] else row['municipio']}\t{row['rs'] if row['rs'] else '#N/D'}\t{row['data_cura_obito'].day}/{static.meses[row['data_cura_obito'].month-1]}/{row['data_cura_obito'].year}\n")
    relatorio.write('\n')

    if True:

        #casos
        relatorio.write(f"{len(novos_casosPR):,} novos casos residentes divulgados no PR.\n".replace(',','.'))

        relatorio.write(f"{len(retroativos):,} casos retroativos confirmados no período de {retroativos.iloc[0]['data_liberacao'].strftime('%d/%m/%Y')} à {retroativos.iloc[-1]['data_liberacao'].strftime('%d/%m/%Y')}.\n".replace(',','.'))
        relatorio.write(f"{len(last2weeks):,} novos casos confirmados no período de {last2weeks.iloc[0]['data_liberacao'].strftime('%d/%m/%Y')} à {last2weeks.iloc[-1]['data_liberacao'].strftime('%d/%m/%Y')}.\n".replace(',','.'))
        relatorio.write(f"{len(hoje):,} novos casos confirmados desde o ultimo informe.\n\n".replace(',','.'))


        novos_casosPR['month'] = novos_casosPR.apply(lambda x: x['data_liberacao'].month, axis=1)
        novos_casosPR['year'] = novos_casosPR.apply(lambda x: x['data_liberacao'].year, axis=1)
        relatorio.write('Novos casos por meses:\n')
        for group, value in novos_casosPR.groupby(by=['year','month']):
            relatorio.write(f"{static.meses[int(group[1])-1]}\\{group[0]}: {len(value)}\n")
        relatorio.write('\n')

        #obitos
        relatorio.write(f"{len(novos_obitosPR):,} novos obitos residentes divulgados no PR.\n".replace(',','.'))

        relatorio.write(f"{len(obitos_retroativos):,} obitos retroativos ocorridos no período de {obitos_retroativos.iloc[0]['data_cura_obito'].strftime('%d/%m/%Y')} à {obitos_retroativos.iloc[-1]['data_cura_obito'].strftime('%d/%m/%Y')}.\n".replace(',','.'))
        relatorio.write(f"{len(obitos_last2weeks):,} novos obitos ocorridos no período de {obitos_last2weeks.iloc[0]['data_cura_obito'].strftime('%d/%m/%Y')} à {obitos_last2weeks.iloc[-1]['data_cura_obito'].strftime('%d/%m/%Y')}.\n".replace(',','.'))
        relatorio.write(f"{len(obitos_hoje):,} novos obitos ocorridos desde o ultimo informe.\n\n".replace(',','.'))

        relatorio.write('Novos obitos por meses:\n')
        novos_obitosPR['month'] = novos_obitosPR.apply(lambda x: x['data_cura_obito'].month, axis=1)
        novos_obitosPR['year'] = novos_obitosPR.apply(lambda x: x['data_cura_obito'].year, axis=1)
        for group, value in novos_obitosPR.groupby(by=['year','month']):
            relatorio.write(f"{static.meses[int(group[1])-1]}\\{group[0]}: {len(value)}\n")
        relatorio.write('\n')

        relatorio.write('Novos obitos por dia:\n')
        for group, value in novos_obitosPR.groupby(by='data_cura_obito'):
            relatorio.write(f"{group.strftime('%d/%m/%Y')}: {len(value)}\n")

        #-----RELATÓRIO DA COMUNICAÇÃO--------------
        obitos_list = []
        munic = []
        for municipio, obitos in novos_obitosPR_group:
            obito = len(obitos)
            obitos_list.append(obito)
            munic.append(municipio)             

        dicionario = (dict(zip(list(munic),list(obitos_list))))
        #print(dicionario)
        dicionario = sorted(dicionario.items(),key=lambda x: x[1], reverse = True)
        #print(dicionario)


        relatorio.write(f"\nOs pacientes que foram a óbito residiam em: ")
        for municip, obit in dict(dicionario).items():
            if obit != 1:
                relatorio.write(f"{municip} ({obit})")
                relatorio.write(f", ")
        relatorio.write(f".\n")
        relatorio.write(f"A Sesa registra ainda a morte de uma pessoa que residia em cada um dos seguintes municípios:")
        for municip, obit in dict(dicionario).items():
            if obit == 1:
                relatorio.write(f"{municip}")
                relatorio.write(f", ")


Total de casos: 1059503 + 1180
Total de obitos: 25481 + 25


Total de casos PR: 1053718 + 1175
Total de obitos PR: 25337 + 25
Total de casos Fora: 5785 + 5
Total de obitos Fora: 144 + 0


In [8]:
with codecs.open(f"relatorio_{(today.strftime('%d/%m/%Y_%Hh').replace('/','_').replace(' ',''))}.txt","r","utf-8-sig") as relatorio:
    print("\nrelatorio:\n")
    print(relatorio.read())


relatorio:

23/05/2021
1.175 novos casos residentes e 5 não residentes divulgados no PR.
1.054.893 casos confirmados residentes do PR.
1.060.683 total geral.

25 Óbitos residentes do PR:
1 Abatiá
1 Andirá
1 Cascavel
1 Congonhinhas
1 Cornélio Procópio
1 Curitiba
2 Dois Vizinhos
2 Foz do Iguaçu
1 Imbaú
1 Jacarezinho
1 Londrina
1 Marilena
1 Maringá
1 Pato Branco
1 Rebouças
1 Ribeirão do Pinhal
1 Salgado Filho
3 Tomazina
3 União da Vitória

25.362 óbitos residentes do PR.
25.506 total geral.

F	76	União da Vitória	6	22/mai/2021
F	89	Cornélio Procópio	18	23/mai/2021
F	66	Londrina	17	21/mai/2021
M	85	Abatiá	18	22/mai/2021
F	59	Pato Branco	7	22/mai/2021
F	55	Curitiba	2	15/mai/2021
F	62	Imbaú	21	1/mai/2021
M	55	Dois Vizinhos	8	23/mai/2021
F	31	Maringá	15	22/mai/2021
M	49	Andirá	18	21/mai/2021
M	57	Marilena	14	22/mai/2021
M	73	Ribeirão do Pinhal	18	22/mai/2021
F	78	União da Vitória	6	16/mai/2021
F	76	Tomazina	19	22/mai/2021
M	63	União da Vitória	6	17/mai/2021
F	32	Dois Vizinhos	8	22/mai/2021
F